# 02. Silver Layer: RA患者定義とデータ変換 (SQL版)

## 概要
Bronzeデータに対して論文のRA定義を適用し、分析用のSilverデータを作成します。
本ノートブックはDuckDBを使用してSQLで処理を実行します。

## 論文の7つのRA定義

| 定義 | DMARDs処方条件 | CS処方条件 | 患者数 | 有病率(%) |
|------|---------------|-----------|--------|----------|
| Definition 0 | 条件なし（ICD-10コードのみ） | 条件なし | 1,116,122 | 0.88 |
| Definition 1 | csDMARDs/bDMARDs/tsDMARDs 1ヶ月以上 | または2ヶ月以上 | 1,026,634 | 0.81 |
| Definition 2 | csDMARDs/bDMARDs/tsDMARDs 1ヶ月以上 | 条件なし | 869,340 | 0.69 |
| **Definition 3** | **csDMARDs/bDMARDs/tsDMARDs 2ヶ月以上** | **条件なし** | **825,772** | **0.65** |
| Definition 4 | csDMARDs/bDMARDs/tsDMARDs 6ヶ月以上 | 条件なし | 583,137 | 0.46 |
| Definition 5 | MTX/SSZ/TAC/BUC/IGT/bDMARDs/tsDMARDs 1ヶ月以上 | 条件なし | 841,599 | 0.66 |
| Definition 6 | MTX/SSZ/TAC/BUC/IGT/bDMARDs/tsDMARDs 2ヶ月以上 | 条件なし | 798,114 | 0.63 |

**本分析ではDefinition 3を採用**（論文と同様）

In [ ]:
import duckdb
import pandas as pd
import os

# データディレクトリ
BRONZE_DIR = "../data/bronze"
SILVER_DIR = "../data/silver"
os.makedirs(SILVER_DIR, exist_ok=True)

# DuckDB接続
con = duckdb.connect()

print("Silver Layer処理を開始します... (SQL版)")

## 1. Bronzeデータの読み込み

In [ ]:
# Bronzeデータをビューとして登録
con.execute(f"CREATE OR REPLACE VIEW patients AS SELECT * FROM read_parquet('{BRONZE_DIR}/patients.parquet')")
con.execute(f"CREATE OR REPLACE VIEW re_receipt AS SELECT * FROM read_parquet('{BRONZE_DIR}/re_receipt.parquet')")
con.execute(f"CREATE OR REPLACE VIEW sy_disease AS SELECT * FROM read_parquet('{BRONZE_DIR}/sy_disease.parquet')")
con.execute(f"CREATE OR REPLACE VIEW iy_medication AS SELECT * FROM read_parquet('{BRONZE_DIR}/iy_medication.parquet')")
con.execute(f"CREATE OR REPLACE VIEW si_procedure AS SELECT * FROM read_parquet('{BRONZE_DIR}/si_procedure.parquet')")
con.execute(f"CREATE OR REPLACE VIEW ho_insurer AS SELECT * FROM read_parquet('{BRONZE_DIR}/ho_insurer.parquet')")

# 各テーブルのレコード数を確認
tables = ['patients', 're_receipt', 'sy_disease', 'iy_medication', 'si_procedure', 'ho_insurer']
print("Bronzeデータを読み込みました:")
for table in tables:
    count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"  - {table}: {count} records")

## 2. RA関連コードの定義

In [ ]:
# RA関連ICD-10コード（論文と同じ定義）
RA_ICD10_CODES = [
    # M05.x: 血清陽性関節リウマチ
    'M050', 'M051', 'M052', 'M053', 'M058', 'M059',
    # M06.x: その他の関節リウマチ（M061とM064を除く）
    'M060', 'M062', 'M063', 'M068', 'M069',
    # M08.x: 若年性関節炎（M081とM082を除く）
    'M080', 'M083', 'M084', 'M088', 'M089'
]

# DMARDs医薬品コード
CSDMARD_CODES = ['1199101', '1199102', '1199201', '1199301', '1199401', '1199501', '1199601']
BDMARD_TNFI_CODES = ['4400101', '4400102', '4400103', '4400104', '4400105']
BDMARD_IL6I_CODES = ['4400201', '4400202']
BDMARD_ABT_CODES = ['4400301']
TSDMARD_JAKI_CODES = ['4400401', '4400402']

# 全DMARDsコード
ALL_DMARD_CODES = (
    CSDMARD_CODES + BDMARD_TNFI_CODES + BDMARD_IL6I_CODES + 
    BDMARD_ABT_CODES + TSDMARD_JAKI_CODES
)

# 経口ステロイドコード
CS_CODES = ['2454001', '2454002', '2454003']

# SQL用にリストを文字列に変換
RA_ICD10_SQL = "('" + "','".join(RA_ICD10_CODES) + "')"
ALL_DMARD_SQL = "('" + "','".join(ALL_DMARD_CODES) + "')"
CSDMARD_SQL = "('" + "','".join(CSDMARD_CODES) + "')"
BDMARD_TNFI_SQL = "('" + "','".join(BDMARD_TNFI_CODES) + "')"
BDMARD_IL6I_SQL = "('" + "','".join(BDMARD_IL6I_CODES) + "')"
BDMARD_ABT_SQL = "('" + "','".join(BDMARD_ABT_CODES) + "')"
TSDMARD_JAKI_SQL = "('" + "','".join(TSDMARD_JAKI_CODES) + "')"
CS_SQL = "('" + "','".join(CS_CODES) + "')"

print(f"RA関連ICD-10コード数: {len(RA_ICD10_CODES)}")
print(f"DMARDsコード数: {len(ALL_DMARD_CODES)}")

## 3. RA関連ICD-10コードを持つ患者の抽出（Definition 0）

In [ ]:
# Definition 0: ICD-10コードのみ
con.execute(f"""
    CREATE OR REPLACE VIEW ra_patients_def0 AS
    SELECT DISTINCT 共通キー
    FROM sy_disease
    WHERE ICD10コード IN {RA_ICD10_SQL}
""")

def0_count = con.execute("SELECT COUNT(*) FROM ra_patients_def0").fetchone()[0]
print(f"Definition 0 (ICD-10のみ): {def0_count} patients")

## 4. DMARDs処方月数の計算

In [ ]:
# DMARDs処方月数を計算するビューを作成
con.execute(f"""
    CREATE OR REPLACE VIEW dmard_prescription_months AS
    SELECT 
        iy.共通キー,
        COUNT(DISTINCT re.診療年月) AS prescription_months
    FROM iy_medication iy
    INNER JOIN re_receipt re ON iy.検索番号 = re.検索番号
    WHERE iy.共通キー IN (SELECT 共通キー FROM ra_patients_def0)
      AND iy.医薬品コード IN {ALL_DMARD_SQL}
    GROUP BY iy.共通キー
""")

# 処方月数の分布を確認
df_dmard_stats = con.execute("""
    SELECT 
        COUNT(*) AS count,
        AVG(prescription_months) AS mean,
        MIN(prescription_months) AS min,
        MAX(prescription_months) AS max,
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY prescription_months) AS q25,
        PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY prescription_months) AS median,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY prescription_months) AS q75
    FROM dmard_prescription_months
""").fetchdf()

print(f"DMARDs処方のある患者数: {int(df_dmard_stats['count'].iloc[0])}")
print(f"\n処方月数の分布:")
print(df_dmard_stats.T)

## 5. 各RA定義の適用

In [ ]:
# Definition 2: DMARDs 1ヶ月以上
con.execute("""
    CREATE OR REPLACE VIEW ra_patients_def2 AS
    SELECT DISTINCT d0.共通キー
    FROM ra_patients_def0 d0
    INNER JOIN dmard_prescription_months dpm ON d0.共通キー = dpm.共通キー
    WHERE dpm.prescription_months >= 1
""")

# Definition 3: DMARDs 2ヶ月以上（主要定義）
con.execute("""
    CREATE OR REPLACE VIEW ra_patients_def3 AS
    SELECT DISTINCT d0.共通キー
    FROM ra_patients_def0 d0
    INNER JOIN dmard_prescription_months dpm ON d0.共通キー = dpm.共通キー
    WHERE dpm.prescription_months >= 2
""")

# Definition 4: DMARDs 6ヶ月以上
con.execute("""
    CREATE OR REPLACE VIEW ra_patients_def4 AS
    SELECT DISTINCT d0.共通キー
    FROM ra_patients_def0 d0
    INNER JOIN dmard_prescription_months dpm ON d0.共通キー = dpm.共通キー
    WHERE dpm.prescription_months >= 6
""")

# 総患者数を取得
total_patients = con.execute("SELECT COUNT(*) FROM patients").fetchone()[0]

# 各定義の患者数を集計
df_definitions = con.execute(f"""
    SELECT 
        'def_0' AS definition,
        (SELECT COUNT(*) FROM ra_patients_def0) AS n_patients,
        (SELECT COUNT(*) FROM ra_patients_def0) * 100.0 / {total_patients} AS prevalence_pct
    UNION ALL
    SELECT 
        'def_2' AS definition,
        (SELECT COUNT(*) FROM ra_patients_def2) AS n_patients,
        (SELECT COUNT(*) FROM ra_patients_def2) * 100.0 / {total_patients} AS prevalence_pct
    UNION ALL
    SELECT 
        'def_3' AS definition,
        (SELECT COUNT(*) FROM ra_patients_def3) AS n_patients,
        (SELECT COUNT(*) FROM ra_patients_def3) * 100.0 / {total_patients} AS prevalence_pct
    UNION ALL
    SELECT 
        'def_4' AS definition,
        (SELECT COUNT(*) FROM ra_patients_def4) AS n_patients,
        (SELECT COUNT(*) FROM ra_patients_def4) * 100.0 / {total_patients} AS prevalence_pct
""").fetchdf()

print("=" * 50)
print("RA定義別の患者数")
print("=" * 50)
for _, row in df_definitions.iterrows():
    print(f"{row['definition']}: {int(row['n_patients']):,} patients ({row['prevalence_pct']:.2f}%)")

## 6. Definition 3に基づくRA患者データの作成

In [ ]:
# RA患者の基本情報と年齢群を計算
con.execute("""
    CREATE OR REPLACE VIEW ra_patients_base AS
    SELECT 
        p.*,
        CASE 
            WHEN p.age BETWEEN 16 AND 19 THEN '16-19'
            WHEN p.age BETWEEN 20 AND 29 THEN '20-29'
            WHEN p.age BETWEEN 30 AND 39 THEN '30-39'
            WHEN p.age BETWEEN 40 AND 49 THEN '40-49'
            WHEN p.age BETWEEN 50 AND 59 THEN '50-59'
            WHEN p.age BETWEEN 60 AND 69 THEN '60-69'
            WHEN p.age BETWEEN 70 AND 79 THEN '70-79'
            WHEN p.age BETWEEN 80 AND 84 THEN '80-84'
            ELSE '85+'
        END AS age_group
    FROM patients p
    WHERE p.共通キー IN (SELECT 共通キー FROM ra_patients_def3)
""")

# 基本統計を表示
df_basic_stats = con.execute("""
    SELECT 
        COUNT(*) AS total_patients,
        SUM(CASE WHEN sex = '2' THEN 1 ELSE 0 END) AS female_count,
        ROUND(SUM(CASE WHEN sex = '2' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS female_pct
    FROM ra_patients_base
""").fetchdf()

print(f"Definition 3 RA患者数: {int(df_basic_stats['total_patients'].iloc[0])}")
print(f"\n性別分布:")
sex_dist = con.execute("SELECT sex, COUNT(*) AS count FROM ra_patients_base GROUP BY sex ORDER BY sex").fetchdf()
print(sex_dist)
print(f"\n女性比率: {df_basic_stats['female_pct'].iloc[0]}%")

## 7. 薬剤使用パターンの集計

In [ ]:
# 各患者の薬剤使用フラグを作成
con.execute(f"""
    CREATE OR REPLACE VIEW drug_usage AS
    SELECT 
        p.共通キー,
        MAX(CASE WHEN iy.医薬品コード IN ('1199101', '1199102') THEN 1 ELSE 0 END) AS MTX,
        MAX(CASE WHEN iy.医薬品コード IN ('1199201') THEN 1 ELSE 0 END) AS SSZ,
        MAX(CASE WHEN iy.医薬品コード IN ('1199401') THEN 1 ELSE 0 END) AS BUC,
        MAX(CASE WHEN iy.医薬品コード IN ('1199301') THEN 1 ELSE 0 END) AS TAC,
        MAX(CASE WHEN iy.医薬品コード IN ('1199501') THEN 1 ELSE 0 END) AS IGT,
        MAX(CASE WHEN iy.医薬品コード IN ('1199601') THEN 1 ELSE 0 END) AS LEF,
        MAX(CASE WHEN iy.医薬品コード IN {BDMARD_TNFI_SQL} THEN 1 ELSE 0 END) AS TNFI,
        MAX(CASE WHEN iy.医薬品コード IN {BDMARD_IL6I_SQL} THEN 1 ELSE 0 END) AS IL6I,
        MAX(CASE WHEN iy.医薬品コード IN {BDMARD_ABT_SQL} THEN 1 ELSE 0 END) AS ABT,
        MAX(CASE WHEN iy.医薬品コード IN {TSDMARD_JAKI_SQL} THEN 1 ELSE 0 END) AS JAKi,
        MAX(CASE WHEN iy.医薬品コード IN {CS_SQL} THEN 1 ELSE 0 END) AS CS
    FROM ra_patients_def3 p
    LEFT JOIN iy_medication iy ON p.共通キー = iy.共通キー
    GROUP BY p.共通キー
""")

# 薬剤使用率を表示
df_drug_rates = con.execute("""
    SELECT 
        ROUND(AVG(MTX) * 100, 1) AS MTX_pct,
        ROUND(AVG(SSZ) * 100, 1) AS SSZ_pct,
        ROUND(AVG(BUC) * 100, 1) AS BUC_pct,
        ROUND(AVG(TAC) * 100, 1) AS TAC_pct,
        ROUND(AVG(IGT) * 100, 1) AS IGT_pct,
        ROUND(AVG(LEF) * 100, 1) AS LEF_pct,
        ROUND(AVG(TNFI) * 100, 1) AS TNFI_pct,
        ROUND(AVG(IL6I) * 100, 1) AS IL6I_pct,
        ROUND(AVG(ABT) * 100, 1) AS ABT_pct,
        ROUND(AVG(JAKi) * 100, 1) AS JAKi_pct,
        ROUND(AVG(CS) * 100, 1) AS CS_pct
    FROM drug_usage
""").fetchdf()

print("薬剤使用率（全RA患者）:")
print("=" * 40)
for col in df_drug_rates.columns:
    drug_name = col.replace('_pct', '')
    print(f"{drug_name}: {df_drug_rates[col].iloc[0]}%")

## 8. 診療行為（手術・検査）の集計

In [ ]:
# 各患者の手術・検査実施フラグを作成
con.execute("""
    CREATE OR REPLACE VIEW procedure_usage AS
    SELECT 
        p.共通キー,
        MAX(CASE WHEN si.procedure_type = 'TJR' THEN 1 ELSE 0 END) AS TJR,
        MAX(CASE WHEN si.procedure_type = 'ARTHROPLASTY' THEN 1 ELSE 0 END) AS ARTHROPLASTY,
        MAX(CASE WHEN si.procedure_type = 'SYNOVECTOMY' THEN 1 ELSE 0 END) AS SYNOVECTOMY,
        MAX(CASE WHEN si.procedure_type = 'ULTRASOUND' THEN 1 ELSE 0 END) AS ULTRASOUND,
        MAX(CASE WHEN si.procedure_type = 'BMD' THEN 1 ELSE 0 END) AS BMD
    FROM ra_patients_def3 p
    LEFT JOIN si_procedure si ON p.共通キー = si.共通キー
    GROUP BY p.共通キー
""")

# 手術・検査実施率を表示
df_proc_rates = con.execute("""
    SELECT 
        ROUND(AVG(TJR) * 100, 2) AS TJR_pct,
        ROUND(AVG(ARTHROPLASTY) * 100, 2) AS ARTHROPLASTY_pct,
        ROUND(AVG(SYNOVECTOMY) * 100, 2) AS SYNOVECTOMY_pct,
        ROUND(AVG(ULTRASOUND) * 100, 2) AS ULTRASOUND_pct,
        ROUND(AVG(BMD) * 100, 2) AS BMD_pct
    FROM procedure_usage
""").fetchdf()

print("手術・検査実施率（全RA患者）:")
print("=" * 40)
for col in df_proc_rates.columns:
    proc_name = col.replace('_pct', '')
    print(f"{proc_name}: {df_proc_rates[col].iloc[0]}%")

## 9. Silverデータの作成と保存

In [ ]:
# RA患者マスタを作成
con.execute("""
    CREATE OR REPLACE VIEW ra_master AS
    SELECT 
        pb.*,
        COALESCE(du.MTX, 0) AS MTX,
        COALESCE(du.SSZ, 0) AS SSZ,
        COALESCE(du.BUC, 0) AS BUC,
        COALESCE(du.TAC, 0) AS TAC,
        COALESCE(du.IGT, 0) AS IGT,
        COALESCE(du.LEF, 0) AS LEF,
        COALESCE(du.TNFI, 0) AS TNFI,
        COALESCE(du.IL6I, 0) AS IL6I,
        COALESCE(du.ABT, 0) AS ABT,
        COALESCE(du.JAKi, 0) AS JAKi,
        COALESCE(du.CS, 0) AS CS,
        COALESCE(pu.TJR, 0) AS TJR,
        COALESCE(pu.ARTHROPLASTY, 0) AS ARTHROPLASTY,
        COALESCE(pu.SYNOVECTOMY, 0) AS SYNOVECTOMY,
        COALESCE(pu.ULTRASOUND, 0) AS ULTRASOUND,
        COALESCE(pu.BMD, 0) AS BMD,
        COALESCE(dpm.prescription_months, 0) AS prescription_months,
        -- bDMARDs使用フラグ
        CASE WHEN COALESCE(du.TNFI, 0) = 1 OR COALESCE(du.IL6I, 0) = 1 OR COALESCE(du.ABT, 0) = 1 
             THEN 1 ELSE 0 END AS bDMARDs,
        -- 全DMARDs使用フラグ
        CASE WHEN COALESCE(du.MTX, 0) = 1 OR COALESCE(du.SSZ, 0) = 1 OR COALESCE(du.BUC, 0) = 1 OR
                  COALESCE(du.TAC, 0) = 1 OR COALESCE(du.IGT, 0) = 1 OR COALESCE(du.LEF, 0) = 1 OR
                  COALESCE(du.TNFI, 0) = 1 OR COALESCE(du.IL6I, 0) = 1 OR COALESCE(du.ABT, 0) = 1 OR
                  COALESCE(du.JAKi, 0) = 1
             THEN 1 ELSE 0 END AS any_DMARD,
        -- RA関連手術フラグ
        CASE WHEN COALESCE(pu.TJR, 0) = 1 OR COALESCE(pu.ARTHROPLASTY, 0) = 1 OR COALESCE(pu.SYNOVECTOMY, 0) = 1
             THEN 1 ELSE 0 END AS any_RA_surgery
    FROM ra_patients_base pb
    LEFT JOIN drug_usage du ON pb.共通キー = du.共通キー
    LEFT JOIN procedure_usage pu ON pb.共通キー = pu.共通キー
    LEFT JOIN dmard_prescription_months dpm ON pb.共通キー = dpm.共通キー
""")

# カラムを確認
columns = con.execute("SELECT * FROM ra_master LIMIT 0").description
col_names = [col[0] for col in columns]
print(f"RA患者マスタのカラム: {col_names}")
print(f"\nレコード数: {con.execute('SELECT COUNT(*) FROM ra_master').fetchone()[0]}")

In [ ]:
# 定義別サマリーを表示
print("RA定義別サマリー:")
print(df_definitions)

In [ ]:
# Silverデータの保存
con.execute(f"COPY (SELECT * FROM ra_master) TO '{SILVER_DIR}/ra_patients_def3.parquet' (FORMAT PARQUET)")
df_definitions.to_parquet(f"{SILVER_DIR}/ra_definitions_summary.parquet", index=False)

# 定義別患者リストの保存
con.execute(f"COPY (SELECT 共通キー FROM ra_patients_def0) TO '{SILVER_DIR}/ra_patients_def_0.parquet' (FORMAT PARQUET)")
con.execute(f"COPY (SELECT 共通キー FROM ra_patients_def2) TO '{SILVER_DIR}/ra_patients_def_2.parquet' (FORMAT PARQUET)")
con.execute(f"COPY (SELECT 共通キー FROM ra_patients_def3) TO '{SILVER_DIR}/ra_patients_def_3.parquet' (FORMAT PARQUET)")
con.execute(f"COPY (SELECT 共通キー FROM ra_patients_def4) TO '{SILVER_DIR}/ra_patients_def_4.parquet' (FORMAT PARQUET)")

print("\nSilverデータを保存しました:")
ra_master_count = con.execute('SELECT COUNT(*) FROM ra_master').fetchone()[0]
print(f"  - ra_patients_def3.parquet: {ra_master_count} records")
print(f"  - ra_definitions_summary.parquet: {len(df_definitions)} records")
print(f"  - ra_patients_def_0.parquet")
print(f"  - ra_patients_def_2.parquet")
print(f"  - ra_patients_def_3.parquet")
print(f"  - ra_patients_def_4.parquet")

## 10. Silver Layer データ品質チェック

In [ ]:
print("=" * 60)
print("Silver Layer データ品質サマリー")
print("=" * 60)

# 基本統計
basic_stats = con.execute("""
    SELECT 
        COUNT(*) AS total_patients,
        SUM(CASE WHEN sex = '2' THEN 1 ELSE 0 END) AS female_count,
        ROUND(SUM(CASE WHEN sex = '2' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS female_pct
    FROM ra_master
""").fetchone()

print(f"\n【Definition 3 RA患者の基本統計】")
print(f"総患者数: {basic_stats[0]:,}")
print(f"女性患者数: {basic_stats[1]:,}")
print(f"女性比率: {basic_stats[2]}%")

# 年齢分布
print(f"\n【年齢分布】")
age_dist = con.execute("""
    SELECT 
        age_group,
        COUNT(*) AS count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 1) AS pct
    FROM ra_master
    GROUP BY age_group
    ORDER BY age_group
""").fetchall()

for group, count, pct in age_dist:
    print(f"  {group}: {count} ({pct}%)")

# 65歳以上
elderly_stats = con.execute("""
    SELECT 
        SUM(CASE WHEN age >= 65 THEN 1 ELSE 0 END) AS elderly_count,
        ROUND(SUM(CASE WHEN age >= 65 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS elderly_pct,
        SUM(CASE WHEN age >= 85 THEN 1 ELSE 0 END) AS very_elderly_count,
        ROUND(SUM(CASE WHEN age >= 85 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS very_elderly_pct
    FROM ra_master
""").fetchone()

print(f"\n【65歳以上の患者割合】")
print(f"  {elderly_stats[0]} ({elderly_stats[1]}%)")

print(f"\n【85歳以上の患者割合】")
print(f"  {elderly_stats[2]} ({elderly_stats[3]}%)")

# 薬剤使用率
print(f"\n【薬剤使用率】")
drug_rates = con.execute("""
    SELECT 
        ROUND(AVG(MTX) * 100, 1) AS MTX,
        ROUND(AVG(SSZ) * 100, 1) AS SSZ,
        ROUND(AVG(BUC) * 100, 1) AS BUC,
        ROUND(AVG(TAC) * 100, 1) AS TAC,
        ROUND(AVG(IGT) * 100, 1) AS IGT,
        ROUND(AVG(LEF) * 100, 1) AS LEF,
        ROUND(AVG(TNFI) * 100, 1) AS TNFI,
        ROUND(AVG(IL6I) * 100, 1) AS IL6I,
        ROUND(AVG(ABT) * 100, 1) AS ABT,
        ROUND(AVG(JAKi) * 100, 1) AS JAKi,
        ROUND(AVG(CS) * 100, 1) AS CS,
        ROUND(AVG(bDMARDs) * 100, 1) AS bDMARDs
    FROM ra_master
""").fetchone()

drug_names = ['MTX', 'SSZ', 'BUC', 'TAC', 'IGT', 'LEF', 'TNFI', 'IL6I', 'ABT', 'JAKi', 'CS', 'bDMARDs']
for name, rate in zip(drug_names, drug_rates):
    print(f"  {name}: {rate}%")

# 手術・検査実施率
print(f"\n【手術・検査実施率】")
proc_rates = con.execute("""
    SELECT 
        ROUND(AVG(TJR) * 100, 2) AS TJR,
        ROUND(AVG(ARTHROPLASTY) * 100, 2) AS ARTHROPLASTY,
        ROUND(AVG(SYNOVECTOMY) * 100, 2) AS SYNOVECTOMY,
        ROUND(AVG(ULTRASOUND) * 100, 2) AS ULTRASOUND,
        ROUND(AVG(BMD) * 100, 2) AS BMD,
        ROUND(AVG(any_RA_surgery) * 100, 2) AS any_RA_surgery
    FROM ra_master
""").fetchone()

proc_names = ['TJR', 'ARTHROPLASTY', 'SYNOVECTOMY', 'ULTRASOUND', 'BMD', 'any_RA_surgery']
for name, rate in zip(proc_names, proc_rates):
    print(f"  {name}: {rate}%")

print("\n" + "=" * 60)
print("Silver Layer 処理完了 (SQL版)")
print("=" * 60)

In [ ]:
# 接続を閉じる
con.close()